## Entities, IDs and entityLookup

### Entities

Many of the objects used in Cato API calls are stored in the Cato back end as **entities**, which consist of:
* an ID
* a name
* a type
* a description
* a set of helper fields which can contain type-specific data

These fields are wrapped up in the API schema's [EntityInfo](https://api.catonetworks.com/documentation/#definition-EntityInfo) type. The entity which represents a Cato site looks like this:

```
{
  "entity": {
    "id": "9668",
    "name": "Edinburgh",
    "type": "site"
  },
  "description": "HEADQUARTERS Home Socket and wifi...",
  "helperFields": {
    "connectionType": "Socket X1500",
    "creationDate": "Jul 5, 2019 1:19:14 PM",
    "description": "Home Socket and wifi...",
    "isHA": "false",
    "lastModified": "Oct 06 2024 5:56:33 PM",
    "operationalStatus": "active",
    "protoID": "1000000004",
    "type": "HEADQUARTERS"
  }
}
```

Although IDs are most commonly used with mutations (API calls which change Cato configuration) there are queries (AI calls which retrieve data) which can use entities to filter input, so there are query use cases which require the ability to look entities up.

### The entityLookup query

Both CMA and the Cato API use the [entityLookup](https://api.catonetworks.com/documentation/#query-entityLookup) query to retrieve one or more entities from a single Cato account based on a type and optional search/filter parameters. The types are defined by the [EntityType](https://api.catonetworks.com/documentation/#definition-EntityType) enum, which means that you will receive an error if you request a type which does not exist.

A common use case for entityLookup is to retrieve a list of sites in an account. A simple example in Python looks like this:

In [ ]:
import json
import os

#
# Helper module 
# https://github.com/catonetworks/data-analytics/blob/main/notebooks/cato.py
#
from cato import API

#
# Create the API connection
#
C = API(os.environ["CATO_API_KEY"])

#
# Define variables and query
#
variables = {
    "accountID": os.environ["CATO_ACCOUNT_ID"],
    "type":"site"
}
query = """
query entityLookup($accountID:ID!, $type:EntityType!) {
  entityLookup(accountID: $accountID, type:$type) {
    total
    items {
      entity{
        id
        name
        type
      }
      description
      helperFields
    }
  }
}"""

#
# Make the query using our helper module
#
success,entities = C.send("entityLookup", variables, query)

#
# If successful, iterate over the list of entities in the response
#
if success:
    print(f'Total entities:{entities["data"]["entityLookup"]["total"]}')
    print(f'Total returned:{len(entities["data"]["entityLookup"]["items"])}')
    if len(entities["data"]["entityLookup"]["items"]) > 0:
        print(f'First entity:\n{json.dumps(entities["data"]["entityLookup"]["items"][0], indent=2)}')
    else:
        print('API call successful but zero entities returned')
else:
    print(f'Error: {entities}')


If the call was successful and there is at least one site in the account whose ID you supplied in the input variables, you should see a site entity printed above.

## Pagination

Unfortunately the behaviour of the API with respect to the number of items returned is inconsistent across the different entity types. For some types (such as **timezone** and **country**) the API will always return the full set of entities in a single response (at time of writing, it returns 546 timezones and 249 countries). For other types (such as **host**) the response will be paginated with a default response limit of 50 entities which can be paginated over in order to retrieve the full set and for at least one type (**city**), the API will return a maximum of 1000 entities regardless of how much pagination is done. This is because entityLookup is used extensively by CMA, so in order to improve the responsiveness for interactive users, pagination is disabled for built-in types used by CMA.

Across all scenarios, the safest way to call entityLookup such that all entities are ultimately received is to use the **limit** and **from** input variables in a loop to keep calling entityLookup until the number of items received matches the **total** output field. For types which can paginate, they will iterate around the loop until all items have been received; for types which do not paginate (such as timezone) all will be received in a single call which still conforms to the total variable.

An entityLookup query which includes the from and limit inputs looks like this:

In [ ]:
query2 = """
query entityLookup($accountID:ID!, $type:EntityType!, $from:Int=0, $limit:Int=50) {
  entityLookup(accountID: $accountID, type:$type from:$from limit:$limit) {
    total
    items {
      entity{
        id
        name
        type
      }
      description
      helperFields
    }
  }
}"""

And a complete example which uses this query in a loop to return all entities of a specific type (regardless of pagination) looks like this:

In [ ]:
items = []
variables["type"] = "site"
variables["from"] = len(items)
iteration = 0
while True:
    success,entities = C.send("entityLookup", variables, query)
    if not success:
        print(f'entityLookup error: {entities}')
    else:
        iteration += 1
        items += entities["data"]["entityLookup"]["items"]
        print(f'Iteration:{iteration} this_iteration:{len(entities["data"]["entityLookup"]["items"])} total_received:{len(items)} total:{entities["data"]["entityLookup"]["total"]}')
        if len(items) >= entities["data"]["entityLookup"]["total"]:
            break
print(f'Total entities:{entities["data"]["entityLookup"]["total"]}')
print(f'Total returned:{len(items)}')
print(f'    Iterations:{iteration}')
